In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# --- 1. SETUP: Learn from TRAIN Data (Scaler & Target Map) ---
print("STEP 1: Learning parameters from TRAIN data...")
try:
    train_values = pd.read_csv('train_values.csv')
    train_labels = pd.read_csv('train_labels.csv')
    train_df = pd.merge(train_values, train_labels, on='building_id')
except FileNotFoundError:
    print("❌ ERROR: 'train_values.csv' or 'train_labels.csv' not found. These are needed to learn encoding maps.")
    exit()

# Features for fitting
X_fit = train_df.drop(['damage_grade', 'building_id'], axis=1)
y_fit = train_df['damage_grade']
global_mean = y_fit.mean() # For filling NaNs in Target Encoding

# Feature Groups
CONT_FEATURES = ['age', 'area_percentage', 'height_percentage']
ORDINAL_FEATURES = ['count_floors_pre_eq', 'count_families']
OHE_FEATURES = [
    'land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 
    'other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status'
]
TARGET_ENCODE_FEATURES = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']
FEATURES_TO_SCALE = CONT_FEATURES + ORDINAL_FEATURES

# A. Fit Scaler (Log age first)
X_fit['age_log'] = np.log1p(X_fit['age'])
X_fit = X_fit.drop('age', axis=1)
FEATURES_TO_SCALE.remove('age')
FEATURES_TO_SCALE.append('age_log')

scaler = StandardScaler()
scaler.fit(X_fit[FEATURES_TO_SCALE])

# B. Fit Target Encoding Maps
target_maps = {}
for col in TARGET_ENCODE_FEATURES:
    # Calculate mean damage_grade for each category in TRAIN
    target_maps[col] = y_fit.groupby(X_fit[col]).mean().to_dict()

# C. Get OHE Columns Structure (to ensure Test has same columns)
X_fit_ohe = pd.get_dummies(X_fit[OHE_FEATURES], drop_first=True)
ohe_train_columns = X_fit_ohe.columns.tolist() # Keep this list!


# --- 2. APPLY TO TEST DATA (Transform Only) ---
print("\nSTEP 2: Applying preprocessing to TEST data...")
try:
    test_values = pd.read_csv('test_values.csv')
except FileNotFoundError:
    print("❌ ERROR: 'test_values.csv' not found.")
    exit()

X_test = test_values.drop('building_id', axis=1)
test_ids = test_values['building_id']

# A. Log Transform & Scaling (Use fitted scaler)
X_test['age_log'] = np.log1p(X_test['age'])
X_test = X_test.drop('age', axis=1)
X_test[FEATURES_TO_SCALE] = scaler.transform(X_test[FEATURES_TO_SCALE])

# B. One-Hot Encoding & Column Alignment
print("-> Applying One-Hot Encoding...")
X_test = pd.get_dummies(X_test, columns=OHE_FEATURES, drop_first=True)

# Important: Align columns with Train set (Add missing cols with 0, drop extra cols)
# 1. Add missing columns
for col in ohe_train_columns:
    if col not in X_test.columns:
        X_test[col] = 0
# 2. Reorder/Filter columns to match Train exactly (excluding other numeric cols for now)
# We will do a full reindex at the end to be safe, but let's handle dimensionality reduction first.

# C. Dimensionality Reduction (Drop redundant cols found in Train analysis)
REDUNDANT_COLS_TO_DROP = ['land_surface_condition_t', 'roof_type_q', 'position_t']
X_test = X_test.drop(columns=REDUNDANT_COLS_TO_DROP, errors='ignore')

# D. Target Encoding (Use learned maps)
print("-> Applying Target Encoding (using Train maps)...")
for col in TARGET_ENCODE_FEATURES:
    # Map values using the Train dictionary
    X_test[f'{col}_target_enc'] = X_test[col].map(target_maps[col])
    # Fill unseen categories (NaNs) with global mean from Train
    X_test[f'{col}_target_enc'] = X_test[f'{col}_target_enc'].fillna(global_mean)
    X_test = X_test.drop(col, axis=1)

# E. Boolean to Integer
bool_cols = X_test.select_dtypes(include=['bool']).columns
if not bool_cols.empty:
    X_test[bool_cols] = X_test[bool_cols].astype(int)

# F. Re-insert ID
X_test.insert(0, 'building_id', test_ids)

# Save
output_filename = 'preprocessed_test_values_final.csv'
X_test.to_csv(output_filename, index=False)
print(f"\n✅ Successfully created '{output_filename}' with {X_test.shape[0]} rows.")
print(f"   (Use this file to generate predictions for submission)")

STEP 1: Learning parameters from TRAIN data...

STEP 2: Applying preprocessing to TEST data...
-> Applying One-Hot Encoding...
-> Applying Target Encoding (using Train maps)...

✅ Successfully created 'preprocessed_test_values_final.csv' with 86868 rows.
   (Use this file to generate predictions for submission)


In [1]:
#마찬가지로 차원 축소 없는 버전.
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# --- 1. SETUP: Learn from TRAIN Data (Scaler & Target Map) ---
print("STEP 1: Learning parameters from TRAIN data...")
try:
    train_values = pd.read_csv('train_values.csv')
    train_labels = pd.read_csv('train_labels.csv')
    train_df = pd.merge(train_values, train_labels, on='building_id')
except FileNotFoundError:
    print("❌ ERROR: 'train_values.csv' or 'train_labels.csv' not found. These are needed to learn encoding maps.")
    exit()

# Features for fitting
X_fit = train_df.drop(['damage_grade', 'building_id'], axis=1)
y_fit = train_df['damage_grade']
global_mean = y_fit.mean() # For filling NaNs in Target Encoding

# Feature Groups
CONT_FEATURES = ['age', 'area_percentage', 'height_percentage']
ORDINAL_FEATURES = ['count_floors_pre_eq', 'count_families']
OHE_FEATURES = [
    'land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 
    'other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status'
]
TARGET_ENCODE_FEATURES = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']
FEATURES_TO_SCALE = CONT_FEATURES + ORDINAL_FEATURES

# A. Fit Scaler (Log age first)
X_fit['age_log'] = np.log1p(X_fit['age'])
X_fit = X_fit.drop('age', axis=1)
FEATURES_TO_SCALE.remove('age')
FEATURES_TO_SCALE.append('age_log')

scaler = StandardScaler()
scaler.fit(X_fit[FEATURES_TO_SCALE])

# B. Fit Target Encoding Maps
target_maps = {}
for col in TARGET_ENCODE_FEATURES:
    # Calculate mean damage_grade for each category in TRAIN
    # Series로 변환하여 저장
    means = y_fit.groupby(X_fit[col]).mean()
    target_maps[col] = means.to_dict()

# C. Get OHE Columns Structure (to ensure Test has same columns)
X_fit_ohe = pd.get_dummies(X_fit[OHE_FEATURES], drop_first=True)
ohe_train_columns = X_fit_ohe.columns.tolist() 


# --- 2. APPLY TO TEST DATA (Transform Only) ---
print("\nSTEP 2: Applying preprocessing to TEST data...")
try:
    test_values = pd.read_csv('test_values.csv')
except FileNotFoundError:
    print("❌ ERROR: 'test_values.csv' not found.")
    exit()

X_test = test_values.drop('building_id', axis=1)
test_ids = test_values['building_id']

# A. Log Transform & Scaling (Use fitted scaler)
X_test['age_log'] = np.log1p(X_test['age'])
X_test = X_test.drop('age', axis=1)
X_test[FEATURES_TO_SCALE] = scaler.transform(X_test[FEATURES_TO_SCALE])

# B. One-Hot Encoding & Column Alignment
print("-> Applying One-Hot Encoding...")
X_test = pd.get_dummies(X_test, columns=OHE_FEATURES, drop_first=True)

# Important: Align columns with Train set (Add missing cols with 0)
# 1. Add missing columns
for col in ohe_train_columns:
    if col not in X_test.columns:
        X_test[col] = 0

# -------------------------------------------------------------
# [삭제됨] Dimensionality Reduction 부분 제거
# -------------------------------------------------------------

# D. Target Encoding (Use learned maps)
print("-> Applying Target Encoding (using Train maps)...")
for col in TARGET_ENCODE_FEATURES:
    # Map values using the Train dictionary
    X_test[f'{col}_target_enc'] = X_test[col].map(target_maps[col])
    # Fill unseen categories (NaNs) with global mean from Train
    X_test[f'{col}_target_enc'] = X_test[f'{col}_target_enc'].fillna(global_mean)
    X_test = X_test.drop(col, axis=1)

# E. Boolean to Integer
bool_cols = X_test.select_dtypes(include=['bool']).columns
if not bool_cols.empty:
    X_test[bool_cols] = X_test[bool_cols].astype(int)

# F. Re-insert ID
X_test.insert(0, 'building_id', test_ids)

# Save
output_filename = 'preprocessed_test_values_final_no_reduction.csv'
X_test.to_csv(output_filename, index=False)
print(f"\n✅ Successfully created '{output_filename}' with {X_test.shape[0]} rows.")
print(f"   (Use this file to generate predictions for submission)")

STEP 1: Learning parameters from TRAIN data...

STEP 2: Applying preprocessing to TEST data...
-> Applying One-Hot Encoding...
-> Applying Target Encoding (using Train maps)...

✅ Successfully created 'preprocessed_test_values_final_no_reduction.csv' with 86868 rows.
   (Use this file to generate predictions for submission)


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# 1. 데이터 준비 (앞서 전처리된 100% 데이터 사용 가정)
# 만약 파일이 없다면 이전 단계의 전처리 코드를 먼저 실행해주세요.
try:
    df = pd.read_csv('final_features_preprocessed_100_percent_no_reduction.csv')
    df_labels = pd.read_csv('train_labels.csv')
    
    # Train 데이터만 추출 (라벨이 있는 데이터)
    train_data = pd.merge(df, df_labels, on='building_id')
    y = train_data['damage_grade'] - 1 # Keras는 0부터 시작하는 클래스를 원하므로 1,2,3 -> 0,1,2로 변환
    X = train_data.drop(['damage_grade', 'building_id'], axis=1)
    
except FileNotFoundError:
    print("❌ 데이터 파일을 찾을 수 없습니다.")
    exit()

# 2. 이진 변수와 수치형 변수 분리
# Boolean 컬럼(0/1로 되어있는 것들)을 찾습니다.
# 주의: 이미 0/1로 변환되어 있으므로, 2개의 고유값을 가진 컬럼을 찾습니다.
binary_cols = [col for col in X.columns if X[col].nunique() == 2]
numerical_cols = [col for col in X.columns if col not in binary_cols]

print(f"이진 변수 개수: {len(binary_cols)}")
print(f"수치형 변수 개수: {len(numerical_cols)}")

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ---------------------------------------------------------
# [모델 1] Entity Embedding을 사용하는 딥러닝 모델
# ---------------------------------------------------------
inputs = []
embeddings = []

# 1) 이진 변수 처리 (Embedding Layer)
for col in binary_cols:
    input_layer = Input(shape=(1,), name=col)
    inputs.append(input_layer)
    # 2개의 범주(0, 1)를 2차원 벡터로 임베딩 (input_dim=2, output_dim=2)
    embedding = Embedding(input_dim=2, output_dim=2, name=f'{col}_emb')(input_layer)
    vec = Flatten()(embedding)
    embeddings.append(vec)

# 2) 수치형 변수 처리 (그대로 입력)
if len(numerical_cols) > 0:
    num_input = Input(shape=(len(numerical_cols),), name='numerical_input')
    inputs.append(num_input)
    embeddings.append(num_input)

# 3) 병합 및 학습
x = Concatenate()(embeddings)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
output = Dense(3, activation='softmax')(x) # 3개 클래스 예측

model_emb = Model(inputs=inputs, outputs=output)
model_emb.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 입력 데이터 준비 (리스트 형태)
def get_keras_input(df):
    input_list = [df[col].values for col in binary_cols]
    if len(numerical_cols) > 0:
        input_list.append(df[numerical_cols].values)
    return input_list

print("\n🚀 Training Model with Entity Embeddings...")
model_emb.fit(get_keras_input(X_train), y_train, validation_data=(get_keras_input(X_val), y_val), epochs=5, batch_size=256, verbose=1)


# ---------------------------------------------------------
# [모델 2] 일반적인 0/1 입력 (비교군)
# ---------------------------------------------------------
# 단순히 모든 데이터를 하나의 덩어리로 넣는 일반적인 방식
input_all = Input(shape=(X_train.shape[1],))
x = Dense(64, activation='relu')(input_all)
x = Dense(32, activation='relu')(x)
output_simple = Dense(3, activation='softmax')(x)

model_simple = Model(inputs=input_all, outputs=output_simple)
model_simple.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("\n🚀 Training Standard Model (No Embedding)...")
model_simple.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=256, verbose=1)

이진 변수 개수: 52
수치형 변수 개수: 8

🚀 Training Model with Entity Embeddings...
Epoch 1/5
815/815 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.7175 - loss: 0.6287 - val_accuracy: 0.7438 - val_loss: 0.5760
Epoch 2/5
815/815 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7448 - loss: 0.5760 - val_accuracy: 0.7485 - val_loss: 0.5666
Epoch 3/5
815/815 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7456 - loss: 0.5725 - val_accuracy: 0.7501 - val_loss: 0.5623
Epoch 4/5
815/815 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7469 - loss: 0.5693 - val_accuracy: 0.7505 - val_loss: 0.5601
Epoch 5/5
815/815 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7469 - loss: 0.5687 - val_accuracy: 0.7462 - val_loss: 0.5703

🚀 Training Standard Model (No Embedding)...
Epoch 1/5
815/815 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7174 - loss: 0.6352 - val_accuracy: 0.7433 - val_loss: 0.5812
Epoch 2/5
815/815 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7435 - loss: 0.5794 - val_accuracy: 0.7472 - val_loss: 0.

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

print("🚀 Entity Embedding Preprocessing Script Started...")

# --- 1. 데이터 로드 ---
print("-> Loading Data...")
try:
    train_values = pd.read_csv('train_values.csv')
    train_labels = pd.read_csv('train_labels.csv')
    test_values = pd.read_csv('test_values.csv')
    
    # 학습 데이터 병합
    train_df = pd.merge(train_values, train_labels, on='building_id')
except FileNotFoundError:
    print("❌ Error: 데이터 파일을 찾을 수 없습니다.")
    exit()

# 타겟 변수 (Keras는 0부터 시작하는 클래스를 원하므로 1,2,3 -> 0,1,2로 변환)
y = train_df['damage_grade'].values - 1
X_train_raw = train_df.drop(['damage_grade', 'building_id'], axis=1)
X_test_raw = test_values.drop(['building_id'], axis=1)
submission_ids = test_values['building_id']

# --- 2. 변수 구분 (범주형 vs 수치형) ---
# 이진 변수 (Binary) - 요청하신 대로 임베딩 대상에 포함
binary_cols = [col for col in X_train_raw.columns if X_train_raw[col].nunique() == 2]

# 고차원 범주형 (Geo Levels) - 임베딩 효과가 가장 큼
geo_cols = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']

# 나머지 범주형 (Object)
object_cols = [col for col in X_train_raw.select_dtypes(include='object').columns]

# 임베딩을 적용할 모든 범주형 변수 리스트
categorical_cols = geo_cols + object_cols + binary_cols

# 수치형 변수 (임베딩 제외)
numerical_cols = [col for col in X_train_raw.columns if col not in categorical_cols]

print(f"-> Categorical Features for Embedding: {len(categorical_cols)}")
print(f"-> Numerical Features: {len(numerical_cols)}")

# --- 3. 데이터 전처리 (Label Encoding & Scaling) ---
# Train/Test를 합쳐서 인코딩 (범주 일치 위함)
combined_df = pd.concat([X_train_raw, X_test_raw], axis=0)

# 범주형: Label Encoding (각 범주를 0 ~ N-1 정수로 변환)
for col in categorical_cols:
    le = LabelEncoder()
    combined_df[col] = le.fit_transform(combined_df[col].astype(str))

# 수치형: 로그 변환 및 스케일링
if 'age' in numerical_cols:
    combined_df['age'] = np.log1p(combined_df['age'])

scaler = StandardScaler()
combined_df[numerical_cols] = scaler.fit_transform(combined_df[numerical_cols])

# 다시 분리
X_train_enc = combined_df.iloc[:len(X_train_raw)]
X_test_enc = combined_df.iloc[len(X_train_raw):]

# --- 4. Keras 모델 구성 (Entity Embedding 학습용) ---
inputs = []
embeddings = []

# 임베딩 차원 설정 함수 (경험적 규칙)
def get_embedding_dim(n_unique):
    if n_unique <= 2: return 1 # 이진 변수는 1차원으로
    return min(50, (n_unique + 1) // 2)

# 각 범주형 변수에 대해 임베딩 레이어 생성
for col in categorical_cols:
    n_unique = combined_df[col].nunique()
    embed_dim = get_embedding_dim(n_unique)
    
    input_layer = Input(shape=(1,), name=f'in_{col}')
    inputs.append(input_layer)
    
    # Embedding Layer: 정수 인덱스를 벡터로 변환
    embedding = Embedding(input_dim=n_unique, output_dim=embed_dim, name=f'emb_{col}')(input_layer)
    vec = Flatten()(embedding)
    embeddings.append(vec)

# 수치형 변수 추가
if len(numerical_cols) > 0:
    num_input = Input(shape=(len(numerical_cols),), name='in_numerical')
    inputs.append(num_input)
    embeddings.append(num_input)

# 모델 구조 결합
x = Concatenate()(embeddings)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
output = Dense(3, activation='softmax')(x) # 3개 클래스 분류

model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# --- 5. 모델 학습 (임베딩 가중치 학습) ---
print("-> Training Neural Network to learn embeddings...")

# 학습용 입력 데이터 딕셔너리 생성
def make_input_dict(df):
    d = {f'in_{col}': df[col].values for col in categorical_cols}
    if len(numerical_cols) > 0:
        d['in_numerical'] = df[numerical_cols].values
    return d

# 데이터 일부를 검증용으로 분할하여 학습
X_tr, X_val, y_tr, y_val = train_test_split(X_train_enc, y, test_size=0.2, random_state=42)

model.fit(make_input_dict(X_tr), y_tr, 
          validation_data=(make_input_dict(X_val), y_val), 
          epochs=5, batch_size=512, verbose=1)

# --- 6. 임베딩 벡터 추출 및 데이터 변환 ---
print("-> Extracting Embeddings and Transforming Data...")

def replace_with_embeddings(df_enc):
    new_df = pd.DataFrame(index=df_enc.index)
    
    # 수치형은 그대로 복사
    for col in numerical_cols:
        new_df[col] = df_enc[col]
        
    # 범주형은 학습된 임베딩 벡터로 교체
    for col in categorical_cols:
        layer_name = f'emb_{col}'
        # 학습된 가중치(벡터 테이블) 가져오기
        weights = model.get_layer(layer_name).get_weights()[0] 
        
        # 데이터(인덱스)를 벡터로 매핑
        indices = df_enc[col].values
        embedded_values = weights[indices]
        
        # 벡터의 각 차원을 별도의 컬럼으로 추가
        embed_dim = weights.shape[1]
        for i in range(embed_dim):
            new_df[f'{col}_emb_{i}'] = embedded_values[:, i]
            
    return new_df

X_train_embedded = replace_with_embeddings(X_train_enc)
X_test_embedded = replace_with_embeddings(X_test_enc)

# --- 7. 저장 ---
print("-> Saving Files...")

# ID 복구
X_train_embedded.insert(0, 'building_id', train_df['building_id'])
X_test_embedded.insert(0, 'building_id', submission_ids)

# [수정됨] damage_grade 컬럼 추가하지 않음
# X_train_embedded['damage_grade'] = y + 1 

X_train_embedded.to_csv('train_embedded.csv', index=False)
X_test_embedded.to_csv('test_embedded.csv', index=False)

print("✅ 'train_embedded.csv' (without label) and 'test_embedded.csv' created successfully!")
print(f"   Train Shape: {X_train_embedded.shape}")
print(f"   Test Shape: {X_test_embedded.shape}")

🚀 Entity Embedding Preprocessing Script Started...
-> Loading Data...
-> Categorical Features for Embedding: 33
-> Numerical Features: 5
-> Training Neural Network to learn embeddings...
Epoch 1/5
408/408 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.7071 - loss: 0.6466 - val_accuracy: 0.7410 - val_loss: 0.5772
Epoch 2/5
408/408 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7522 - loss: 0.5459 - val_accuracy: 0.7438 - val_loss: 0.5720
Epoch 3/5
408/408 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7599 - loss: 0.5243 - val_accuracy: 0.7439 - val_loss: 0.5755
Epoch 4/5
408/408 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7675 - loss: 0.5065 - val_accuracy: 0.7434 - val_loss: 0.5833
Epoch 5/5
408/408 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7751 - loss: 0.4891 - val_accuracy: 0.7417 - val_loss: 0.5932
-> Extracting Embeddings and Transforming Data...


C:\Users\mzonl\AppData\Local\Temp\ipykernel_45216\1310952388.py:146: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[f'{col}_emb_{i}'] = embedded_values[:, i]
C:\Users\mzonl\AppData\Local\Temp\ipykernel_45216\1310952388.py:146: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[f'{col}_emb_{i}'] = embedded_values[:, i]
C:\Users\mzonl\AppData\Local\Temp\ipykernel_45216\1310952388.py:146: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

-> Saving Files...
✅ 'train_embedded.csv' (without label) and 'test_embedded.csv' created successfully!
   Train Shape: (260601, 165)
   Test Shape: (86868, 165)
